# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Principal Component Analysis

> Authors: Justin Pounders, Matt Brems

### LEARNING OBJECTIVES
By the end of the lesson, students should be able to:
1. Differentiate between feature elimination and feature extraction.
2. Describe the PCA algorithm.
3. Implement PCA in `scikit-learn`.
4. Calculate and interpret proportion of explained variance.
5. Identify use cases for PCA.

In [ ]:
# Import our libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import from sklearn.
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

# Set a random seed.
np.random.seed(42)

%matplotlib inline

### Introduction of Problem

Today, we're going to be using the [wine quality](http://www3.dsi.uminho.pt/pcortez/wine/) dataset by Cortez, Cerdeira, Almeida, Matos and Reis.

Specifically, we are going to use physicochemical properties of the wine in order to predict the quality of the wine.

In [ ]:
# Read in the wine quality datasets.
df_red = pd.read_csv('./datasets/winequality-red.csv', sep=';')
df_white = pd.read_csv('./datasets/winequality-white.csv', sep=';')

# Stack datasets together. (They have the same column names!)
df =

# Check out head of our dataframe.
df.head()

In [ ]:
# Is this a regression problem or classification problem?
round(df['quality'].value_counts(normalize = True), 3) * 100

### Fit a multiple linear regression model in `sklearn`.

In [ ]:
# Set y to be the quality column.
y = df['quality']

# Set X as all other columns.
X = df.drop(columns=['quality'])

# How much missing data do we have?
X.isnull().sum()

In [ ]:
# To show off the strength of PCA, we're going to make many, many more features.
pf = PolynomialFeatures(degree = 3)

# Fit and transform our X data using Polynomial Features.
X_new = pf.fit_transform(X)

# How many features do we have now?
print(X_new.shape)

# How many features did we start out with?
print(df.shape)

In [ ]:
# Train/test split our data.
X_train, X_test, y_train, y_test = train_test_split(X_new,
                                                    y,
                                                    test_size = 0.33,
                                                    random_state = 42)

In [ ]:
# Instantiate and fit a linear regression model.
lm = LinearRegression()
lm.fit(X_train, y_train)

# Score on training set. (We'll use R^2 for the score today.)
print(f'Training Score: {round(lm.score(X_train, y_train),4)}.')

# Score on testing set.
print(f'Testing Score: {round(lm.score(X_test, y_test),4)}.')

<details><summary>Check: What is the problem with this?</summary>
    
- We've clearly overfit our model to the data (so much so that our model's performance is really bad)!
- We have a lot of columns relative to our number of rows! (If you have $n$ rows, it's often advised to keep your number of columns below $\sqrt{n}$.)
</details>

<details><summary>Check: How can we overcome this problem?</summary>

- We can drop features from our model. (However, this loses any benefit we'd get from dropping those features! It can also be time-consuming and/or require subject-matter expertise.)
- Maybe we can combine features together so that we can get the benefits of most/all of our features. (This is what PCA will do.)
</details>

### Dimensionality Reduction

[Dimensionality reduction](https://www.analyticsvidhya.com/blog/2015/07/dimension-reduction-methods/) refers to (approximately) reducing the number of features we use in our model.

<details><summary>Dimensionality reduction has a number of advantages:</summary>

- Increases computational efficiency when fitting models.
- Can help with addressing a multicollinearity problem.
- Makes visualization simpler (or feasible).
</details>

<details><summary>Dimensionality reduction can suffer from some drawbacks, though:</summary>

- We've invested our time and money into collecting information... why do we want to get rid of it?
</details>

### Is there a way to get the advantages of dimensionality reduction while minimizing the drawbacks?

Dimensionality reduction can generally be broken down into one of two categories:
- **Feature Elimination**
    - In feature elimination, we drop variables from our model.
- **Feature Extraction**
    - In feature extraction, we take our existing features and combine them together in a particular way. We can then drop some of these "new" variables, but the variables we keep are still a combination of the old variables!
    - This allows us to still reduce the number of features in our model **but** we can keep all of the most important pieces of the original features!

### $$
\begin{eqnarray*}
X_1, \ldots, X_p &\Rightarrow& Z_1, \ldots, Z_p \\
\\
\text{most important: }Z_1 &=& w_{1,1}X_1 + w_{1,2}X_2 + \cdots + w_{1,p}X_p \\
\text{slightly less important: }Z_2 &=& w_{2,1}X_1 + w_{2,2}X_2 + \cdots + w_{2,p}X_p \\
&\vdots&\\
\text{least important: }Z_p &=& w_{p,1}X_1 + w_{p,2}X_2 + \cdots + w_{p,p}X_p \\
\end{eqnarray*}
$$

- We don't usually care about the values of weights here. They aren't very meaningful and we don't try to interpret them.
- You can think of $Z_1$ as a "high performance" predictor, where $Z_1$ has all of the best pieces of our original predictors $X_1$ through $X_p$.
- As we move down the list toward $Z_p$, the variables will consist of the more "redundant" parts of our $X$ variables. 
- You can think of $Z_p$ as a "low performance" predictor.

I like to use a feast or buffet analogy here.

<details><summary>If I'm going to keep three of my new predictors, which three would I keep?</summary>
    
- The first three: $Z_1$, $Z_2$, and $Z_3$.
- This is where feature extraction comes into play! You can think of $Z_1$, $Z_2$, and $Z_3$ as having takena three-dimensional representation
</details>

Dimensionality reduction can be used as an exploratory/unsupervised learning method or as a pre-processing step for supervised learning later.

**Principal component analysis** is a method used for feature extraction.

<details><summary>How would you describe the difference between feature elimination and feature extraction?</summary>

- Feature elimination is a process of dropping original features from our model.
- Feature extraction is a process of transforming our original features into "new" features, then dropping some of the "new" features from our model.
</details>

## Principal Component Analysis

### Big picture, what is PCA doing?
1. We are going to look at how all of the $X$ variables relate to one another and summarize these relationships.
2. Then, we will take this summary and look at which combinations of our $X$ variables are most important.
3. We can also quantify how important each combination is and rank these combinations.

Once we've taken our original $X$ data and transformed it into $Z$, we can then drop the columns of $Z$ that are "least important."

**Essentially...**

- PCA finds *linear combinations* of current predictor variables that...
- create new "principal components," where these principal components explain...
- the maximum possible amount of variance in your predictors.

$$ Z_1 = w_{1,1}X_1 + w_{1,2}X_2 + \cdots + w_{1,p}X_p $$

$$ Z_2 = w_{2,1}X_1 + w_{2,2}X_2 + \cdots + w_{2,p}X_p $$

$$\vdots$$

$$ Z_p = w_{p,1}X_1 + w_{p,2}X_2 + \cdots + w_{p,p}X_p $$

This is cool because...

- $Z_1$ explains more variability in our $X$ data than $Z_2$, which explains more variability in our $X$ data than $Z_3$, and so on.
- $Z_1$, $Z_2$, $Z_3$, $\ldots$ are *uncorrelated* with one another.

---

**Visually...**

> Think of our data floating out in $p$-dimensional space. Each observation is a dot and you can imagine this massive cloud of dots that exists somewhere. PCA is a way to rotate this cloud of dots (formally, a [coordinate transformation](http://farside.ph.utexas.edu/teaching/336k/Newtonhtml/node153.html)). The old axes are the original $X_1$, $X_2$, $\ldots$ features. **The new axes are the principal components from PCA**.

The principal components are the most concise, informative descriptors of our data as a whole.
- What does this mean?
- If we wanted to take our full data set and condense it into one dimension (think like our $X$ axis), we'd only use $Z_1$.
- If we wanted to take our full data set and condense it into two dimensions (think like our $X$ and $Y$ axes), we'd use $Z_1$ and $Z_2$.

Let's head to [this site](http://setosa.io/ev/principal-component-analysis/). Play around with the 2D data. Take 2-3 minutes.
1. As you interact with the data, how would you describe the red line?
2. As you interact with the data, how would you describe the green line?

---

### Principal Components

- We are looking for new *directions*. ([Insert Glee joke here](https://glee.fandom.com/wiki/New_Directions).)
- Each consecutive direction tries to explain the maximum *remaining variance* in our $X$ data.
- Each direction is *orthogonal* to all the others.

**These new *directions* are the "principal components."**

> Applying PCA to your data *transforms* your original data columns (variables) onto the new principal component axes.


### Two notes:

1. You will do your train/test split **before** applying PCA!
2. Standardize our data **before** applying PCA!

In [ ]:
# Instantiate our StandardScaler.
ss = StandardScaler()

# Scale X_train.
X_train = ss.fit_transform(X_train)

# Scale X_test.
X_test = ss.transform(X_test)

In [ ]:
# Import PCA.


#### (BONUS) Why decomposition?
The way PCA works "under the hood" is it takes one matrix and **decomposes** that matrix into multiple matrices.

Written out, we might take some matrix $\mathbf{A}$ and break it down into matrices $\mathbf{B}$ and $\mathbf{C}$ like this:

$$
\begin{eqnarray*}
\mathbf{A} &=& \mathbf{B}\mathbf{C}
\end{eqnarray*}
$$

This might seem like a silly thing to do, but it actually can be **very** computationally appealing! Check out [the Wikipedia article](https://en.wikipedia.org/wiki/Matrix_decomposition) for a list of 100+ ways to decompose matrices.
- The specific method of decomposition commonly used for PCA is known as the [eigendecomposition](https://en.wikipedia.org/wiki/Eigendecomposition_of_a_matrix) or spectral decomposition of a matrix.

In [ ]:
# Instantiate PCA.
pca =

In [ ]:
# Fit PCA on the training data.


In [ ]:
# Transform PCA on the training data.
Z_train =

In [ ]:
# Let's check out the resulting data.
pd.DataFrame(Z_train).describe()

In [ ]:
# Don't forget to transform the test data!
Z_test = pca.transform(X_test)

### So, like, big picture, what is PCA doing?
Well, we're transforming our data. Specifically, we are:
1. We are going to look at how all of the $X$ variables relate to one another, then summarize these relationships. (This is done with the **covariance matrix**.)
2. Then, we will take this summary and look at which combinations of our $X$ variables are most important. (We will decompose our covariance matrix into its **eigenvectors**, which is a linear algebra term that allows us to understand the most important "directions" in our data, which are our principal components!)
3. We can also see exactly how important each combination is, then rank these combinations. (With each eigenvector, we get an **eigenvalue**. This eigenvalue is a number that tells us how important each "direction" or principal component is.)
    - Want a better understanding of eigenvectors and eigenvalues? [Check this 3Blue1Brown video out!](https://www.youtube.com/watch?v=PFDu9oVAE-g)

Remember that one of our goals with PCA is to do dimensionality reduction (a.k.a. get rid of features).

We can: 
- measure how important each principal component is using the eigenvalue, 
- rank the columns of `Z_train` by their eigenvalues,
- then drop the columns with small eigenvalues (little importance) but keep the columns with big eigenvalues (very important).
    - In `sklearn`, when transformed by PCA, the columns will already be sorted by their eigenvalues from biggest to smallest! The first column will be the most important, the second column will be the next most important, and so on.

#### But how many features do we discard?

A useful measure is the **proportion of explained variance**, which is calculated from the **eigenvalues**. 

The explained variance tells us how much information (variance) is captured by each principal component.

### $$ \text{explained variance of }PC_k = \bigg(\frac{\text{eigenvalue of } PC_k}{\sum_{i=1}^p\text{eigenvalue of } PC_i}\bigg)$$

Rather than write out "$\text{eigenvalue of } PC_k$", we usually just write $\lambda_k$.

If I want to calculate the proportion of explained variance by retaining $PC_1$ and $PC_2$, I would calculate this as:

### $$ \text{explained variance of } PC_1 \text{ and } PC_2 = \bigg(\frac{\lambda_1 + \lambda_2}{\sum_{i=1}^p \lambda_i} \bigg)$$

In [ ]:
# Pull the explained variance attribute.
var_exp =
print(f'Explained variance (first 20 components): {}')

print('')

# Generate the cumulative explained variance.
cum_var_exp =
print(f'Cumulative explained variance (first 20 components): {}')

<details><summary>Check: If I wanted to explain at least 80% of the variability in my data with principal components, what is the smallest number of principal components that I would need to keep? </summary>

- Only six! 
- I could keep $Z_1, Z_2, \ldots, Z_6$ in my model, and this would explain 80.833% of the variability in my $X$ data.
</details>

In [ ]:
# Plot the variance explained (and cumulative variance explained).

# Set figure size.
plt.figure(figsize=(12,8))

# Plot the explained variance.
plt.plot(range(len(var_exp)), var_exp, lw=3, label = 'Variance Explained')

# Plot the cumulative explained variance.
plt.plot(range(len(var_exp)), cum_var_exp, lw=3, color = 'orange', label = 'Cumulative Variance Explained')

# Add horizontal lines at y=0 and y=1.
plt.axhline(y=0, linewidth=1, color='grey', ls='dashed')
plt.axhline(y=1, linewidth=1, color='grey', ls='dashed')

# Set the limits of the axes.
plt.xlim([-1,21])
plt.ylim([-0.01,1.01])

# Label the axes.
plt.ylabel('Variance Explained', fontsize=20)
plt.xlabel('Principal Component', fontsize=20)

# Make the tick labels bigger
plt.xticks(range(0, 21, 5), range(1, 22, 5), fontsize=12)
plt.yticks(fontsize=12)
    
# Add title and legend.
plt.title('Component vs. Variance Explained', fontsize=24)
plt.legend(fontsize=11);

## Let's compare our PCA'ed performance to our original performance!

#### Original performance:

In [ ]:
# Instantiate linear regression model.
lm = LinearRegression()

# Fit on X_train.
lm.fit(X_train, y_train)

# Score on training and testing sets.
print(f'Training Score: {round(lm.score(X_train, y_train),4)}.')
print(f'Testing Score: {round(lm.score(X_test, y_test),4)}.')

<details><summary>What's going on here?!</summary>
    
- This model is really, really bad. 
- It's so overfit that our test $R^2$ is negative!
- When our test $R^2$ is negative, this means that our predicted values $\hat{Y}$ are worse than if we simply predicted the average $Y$ value from our test set.
</details>

#### Principal Component Regression performance:

In [ ]:
# Instantiate PCA with 10 components.
pca =

# Fit PCA to training data.
pca.fit(X_train)

In [ ]:
# Instantiate linear regression model.
lm = LinearRegression()

# Transform Z_train and Z_test.
Z_train = pca.transform(X_train)
Z_test = pca.transform(X_test)

# Fit on Z_train.
lm.fit(Z_train, y_train)

# Score on training and testing sets.
print(f'Training Score: {round(lm.score(Z_train, y_train),4)}')
print(f'Testing Score: {round(lm.score(Z_test, y_test),4)}')

Our final model here is:

$$
\begin{eqnarray*}
[\text{quality}] &=& \beta_0 + \beta_1Z_1 + \beta_2Z_2 + \cdots + \beta_{10}Z_{10} \\
\\
\text{where } Z_1 &=& \gamma_1X_1 + \gamma_2X_2 + \gamma_3X_3 + \cdots \gamma_pX_p \\
\text{and } Z_2 &=& \delta_1X_1 + \delta_2X_2 + \delta_3X_3 + \cdots \delta_pX_p \\
&\vdots& \\
\text{and } Z_{10} &=& \eta_1X_1 + \eta_2X_2 + \eta_3X_3 + \cdots \eta_pX_p \\
\end{eqnarray*}
$$

**Two assumptions that PCA makes:**
1. **Linearity:** PCA detects and controls for linear relationships, so we assume that the data does not hold nonlinear relationships (or that we don't care about these nonlinear relationships).
    - We are using our covariance matrix to determine important "directions," which is a measure of the linear relationship between observations!
    - There are other types of feature extraction like [t-SNE](https://lvdmaaten.github.io/tsne/) and [PPA](https://towardsdatascience.com/interesting-projections-where-pca-fails-fe64ddca73e6), though we won't formally cover those in a global lesson.
    
    
2. **Large variances define importance:** If data is spread in a direction, that direction is important! If there is little spread in a direction, that direction is not very important.
    - That aligns with what we saw [here](http://setosa.io/ev/principal-component-analysis/).

### Potential Use Cases for PCA
- Situations where $p \not\ll n$. (Situations where $p$ is not substantially smaller than $n$.)
- Situations in which there are variables with high multicollinearity. (Can be traditional models or models with highly correlated inputs by design, like images.)
- Situations in which there are many variables, even without explicit multicollinearity.

### Interview Questions

<details><summary>Explain PCA to me.</summary>

- Principal component analysis is a method of dimensionality reduction that **identifies important relationships** in our data, **transforms the existing data** based on these relationships, and then **quantifies the importance** of these relationships so we can keep the most important relationships and drop the others!

<details><summary>How can I remember the above?</summary>

Matt's "Three Signposts:"
- Covariance Matrix
- Eigenvectors
- Eigenvalues
</details>
</details>

<details><summary>In what cases would I not use PCA?</summary>

- Since PCA distorts the interpretability of our features, we should not use PCA if our goal is to interpret the output of our model.
- If we have relatively few features as inputs, PCA is unlikely to have a large positive impact on our model.
</details>

### LEARNING OBJECTIVES
By the end of the lesson, students should be able to:
1. Differentiate between feature elimination and feature extraction.
2. Describe the PCA algorithm.
3. Implement PCA in `scikit-learn`.
4. Calculate and interpret proportion of explained variance.
5. Identify use cases for PCA.